<a href="https://colab.research.google.com/github/ByeongKeun/Industrial-AI/blob/master/2021.1%ED%95%99%EA%B8%B0/%EC%82%B0%EC%97%85%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B0%9C%EB%A1%A0/%EC%8B%A4%EC%8A%B5/2021_05_13_PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Convolution Neural network**

In [1]:
# -*- coding: utf-8 -*-
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist.data/255
y = mnist.target

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

X_train = X_train.view(-1, 1, 28, 28).float()
X_test = X_test.view(-1, 1, 28, 28).float()
print("X_train.shape: ", X_train.shape)
print("X_test.shape: ", X_test.shape)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)

BATCH_SIZE = 32

loader_train = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
loader_test = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

X_train.shape:  torch.Size([60000, 1, 28, 28])
X_test.shape:  torch.Size([10000, 1, 28, 28])


In [12]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
    self.fc1 = nn.Linear(3*3*64, 256)
    self.fc2 = nn.Linear(256, 10)

    self.loss_fn =nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters(), lr=0.01)


def forward(self, x):
  x = F.relu(self.conv1(x))
  x = F.relu(F.max_pool2d(self.conv2(x), 2))
  x = F.dropout(x, p=0.5, training = self.training)
  x = F.relu(F.max_pool2d(self.conv3(x), 2))
  x = F.dropout(x, p=0.5, training = self.training)
  x = x.view(-1, 3*3*64)
  x = F.relu(self.fc1(x))
  x = F.dropout(x, training = self.training)
  x = self.fc2(x)

  return F.log_softmax(x, dim=1)


torch.nn.Conv2d(in_channels=28*28, out_channels=10, kernel_size=5, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

def fit(model, loader_train):
  optimizer = torch.optim.Adam(model.parameters())
  error = nn.CrossEntropyLoss()  
  EPOCHS = 1

  model.train()

  for epoch in range(EPOCHS):
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(loader_train):
      var_X_batch = Variable(X_batch).float()
      var_y_batch = Variable(y_batch)

      optimizer.zero_grad()
      output = model(var_X_batch)
      loss = error(output, var_y_batch)
      loss.backward()
      optimizer.step()
      predicted = torch.max(output.data, 1)[1]
      correct += (predicted == var_y_batch).sum()

      if batch_idx % 50 == 0:
        print('에포크 : {} [{}/{} ({:.0f}%)]\t 손실함수 : {:.6f}\t Accuracy : {:.3f}%'.format(epoch, 
                                                                                       batch_idx*len(X_batch), 
                                                                                       len(loader_train), 
                                                                                       100.*batch_idx / len(loader_train), 
                                                                                       loss.data, 
                                                                                       correct*100./(BATCH_SIZE*(batch_idx+1))))

  def evaluate(model):
    correct = 0
    for test_imgs, test_labels in loader_test:
      test_imgs = Variable(test_imgs).float()
      output = model(test_imgs)
      predicted = torch.max(output, 1)[1]
      correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) / (len(loader_test)*BATCH_SIZE)))

In [13]:
cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval() #모델 테스트 모드로 전환
evaluate(cnn)

NotImplementedError: ignored

In [ ]:
index = 10 #테스트 데이터 중에서 확인해볼 데이터의 인덱스
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data) #모델 적용
print('{}번째 학습데이터의 테스트 결과 : {}'.format(index, output))
_, predicted = torch.max(output, 1)
print('{}번째 데이터의 예측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))